In [1]:
# import
import os
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from scipy.spatial import distance
from sklearn.cluster import KMeans
from tqdm import tqdm

# import plotting libraries
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 10})
plt.rcParams['svg.fonttype'] = 'none'
import seaborn as sns
from nilearn import datasets
from nilearn import plotting

# import network_control functions
from network_control.energies import integrate_u, get_control_inputs
from network_control.pipelines import ComputeControlEnergy, ComputeOptimizedControlEnergy
from network_control.metrics import ave_control
from network_control.utils import matrix_normalization, convert_states_str2float, normalize_state, normalize_weights, \
    get_null_p, get_fdr_p
from network_control.plotting import roi_to_vtx, null_plot, surface_plot
from null_models.geomsurr import geomsurr

# Using get_control_inputs

This notebook covers examples of different ways in which get_control_inputs can used to derive control energy

### Load A

In [2]:
# directory where data is stored
datadir = '/Users/lindenmp/Google-Drive-Penn/work/research_projects/control_package/data'
resultsdir = '/Users/lindenmp/Google-Drive-Penn/work/research_projects/control_package/results'
A_file = 'pnc_schaefer200_Am.npy'

# load A matrix
A = np.load(os.path.join(datadir, A_file))
n_nodes = A.shape[0]
A[np.eye(A.shape[0]).astype(bool)] = 0
print(A.shape)

(200, 200)


### Get initial and target state

In [3]:
# load node-to-system mapping
system_labels = list(np.loadtxt(os.path.join(datadir, 'pnc_schaefer200_system_labels.txt'),
                                dtype=str))

# use list of system names to create states
states, state_labels = convert_states_str2float(system_labels)

# extract initial state
x0 = states == state_labels.index('Vis')

# extract target state
xf = states == state_labels.index('Default')

# normalize state magnitude
x0 = normalize_state(x0)
xf = normalize_state(xf)

# Get energy

By default, we compute $x$ and $u$ using the following settings:
1. Time system: continuous
2. Time horizon: 1
3. Control set: uniform full
4. Amplitude of $x$ and $u$ are constrained (i) toward zero activity; (ii) equally between $x$ and $u$; and (iii) across all nodes

You can compute $x$ and $u$ using the above settings like so:

In [4]:
system = 'continuous'  # choose time system
A_norm = matrix_normalization(A, system=system, c=1)  # normalize A
B = np.eye(n_nodes)  # uniform full control set
T = 1  # time horizon
xr = np.zeros(n_nodes)  # x and u constrained toward zero activity
rho = 1  # x and u constrained equally
S = np.eye(n_nodes)  # x and u constrained over all nodes

# get the state trajectory (x) and the control inputs (u)
x, u, n_err = get_control_inputs(A_norm=A_norm, T=T, B=B, x0=x0, xf=xf, system=system, xr=xr, rho=rho, S=S)
node_energy = integrate_u(u)
energy = np.sum(node_energy)

print('energy: {0:.2f}'.format(energy))

energy: 2638.03


You want to use a discrete time system instead?

In [5]:
system = 'discrete'  # choose time system  <-- NOTE CHANGE
A_norm = matrix_normalization(A, system=system, c=1)  # normalize A
B = np.eye(n_nodes)  # uniform full control set
T = 10  # time horizon
xr = np.zeros(n_nodes)  # x and u constrained toward zero activity
rho = 1  # x and u constrained equally
S = np.eye(n_nodes)  # x and u constrained over all nodes

# get the state trajectory (x) and the control inputs (u)
x, u, n_err = get_control_inputs(A_norm=A_norm, T=T, B=B, x0=x0, xf=xf, system=system, xr=xr, rho=rho, S=S)
node_energy = integrate_u(u)
energy = np.sum(node_energy)

print('energy: {0:.2f}'.format(energy))

energy: 0.51


You want to only constrain u and not x?

Note, this is what we refer to as 'minimum control energy', the energy where only the control signals are constrained

In [6]:
system = 'continuous'  # choose time system
A_norm = matrix_normalization(A, system=system, c=1)  # normalize A
B = np.eye(n_nodes)  # uniform full control set
T = 1  # time horizon
xr = np.zeros(n_nodes)  # x and u constrained toward zero activity
#  <-- NOTE REMOVAL OF RHO
S = np.zeros((n_nodes, n_nodes))  # x and u constrained over all nodes  <-- NOTE CHANGE

# get the state trajectory (x) and the control inputs (u)
x, u, n_err = get_control_inputs(A_norm=A_norm, T=T, B=B, x0=x0, xf=xf, system=system, xr=xr, rho=rho, S=S)  # <-- NOTE REMOVAL OF RHO
node_energy = integrate_u(u)
energy = np.sum(node_energy)

print('energy: {0:.2f}'.format(energy))

energy: 2604.08
